# **Random Diffusion in 2D**

<i class="fa fa-home fa-2x"></i><a href="../index.ipynb" style="font-size: 20px"> Go back to index</a>

**Source code:** https://github.com/osscar-org/quantum-mechanics/blob/develop/notebook/quantum-mechanics/diffusion-from-random-2d.ipynb

<hr style="height:1px;border:none;color:#cccccc;background-color:#cccccc;" />

## Goals

* Understand
    * asd

## Background theory

<details close>
<summary style="font-size: 20px"><b>Random walk</b></summary>
asdasd
  
<div class="container" style="text-align: center; width: 250px;">
  <img src="images/Ising_NN.svg" alt="Nearest neighbours for a spin in a 2D lattice" class="image">
  <div class="overlay">In 2D, the gray spin has 4 nearest neighbours</div>
</div>

asd
</details>

## **Tasks and exercises**

1. Run the simulation with the default parameters (and $J>0$) and check the plot of the magnetization per spin as a function of the simulation step. What happens at very low temperatures? And at intermediate temperatures? And at very high temperatures? And what happens for $J<0$?

    <details>
    <summary><b>Hints</b></summary>
  For a positive $J$ value, parallel neighboring spins are energetically favoured.
  Therefore, at $T=0$ we expect all spins to be aligned: the system is ferromagnetic.
  At low temperature, spin will still tend to be aligned with their neihbors, with some
  local fluctuations as temperature increases.
  Fluctuations become very large close to $T_c$, and above $T_c$
  they become predominant and, on average, we obtain zero net magnetization.<br>

  On the opposite, $J<0$ will lead to a checkerboard pattern for the final spin 
  configuration at low temperatures, i.e., an antiferromagnetic configuration.<br>
    
  When $J=0$, there is no interaction between spins and we always obtain a random spin configuration.<br><br>
    </details>

In [38]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import ipywidgets as ipw
%matplotlib widget
plt.rcParams['figure.autolayout'] = 'True' # turn on tight layout globally

In [39]:
N = 1000
box_xrange = (-10, 10)
box_yrange = (-10, 10)
starting_radius = 0.1
num_steps = 10000

In [40]:
func_drpdwn = ipw.Dropdown(options=['Continuous', 'Discrete'], value='Discrete', description='Random walk type', disabled=False) # continuous or discrete step
n_slider = ipw.IntSlider(value=5000, min=1, max=10000, step=100, description='Number of points') # number of points
max_slider = ipw.FloatSlider(value=0.02, min=0.01, max=0.1, step=0.01,description='Max step size', continuous_update=False, readout=True, readout_format='.2f') # max step size
frame_slider = ipw.IntSlider(value=0, min=0, max=num_steps, step=100, description='Frame index', continuous_update=False, readout=True, disabled=True) # manual control for now, may change to autoplay if it is fast enough
steps_slider = ipw.IntSlider(value=10000, min=100, max=20000, step=100, description='Number of steps', continuous_update=False, disabled=False, layout={})

traj_chkbox = ipw.Checkbox(value=False,description='Show trajectory', disabled=False, indent=False)
map_chkbox = ipw.Checkbox(value=False,description='Show density map', disabled=False, indent=False)

run_btn = ipw.Button(description='Run')
play = ipw.Play(value=0, min=0, max=num_steps, step=100, disabled=True, interval=500) # iterate frame with 500ms interval

In [41]:
# # Run simulations
# import tqdm
# num_steps = 10000
# random_step=0.1

# trajectory = [coords]
# for i in tqdm.tqdm(range(num_steps)):
#     random_displacement = (np.random.random((N, 2)) - 0.5)*2 * random_step
#     new_positions = trajectory[-1] + random_displacement
#     # Some points might have gone beyond the box.
#     # I could either reflect them back as a hard wall, or just use PBC. For simplicity, I use PBC
#     new_positions[:,0] = (new_positions[:,0] - box_xrange[0]) % (box_xrange[1] - box_xrange[0]) + box_xrange[0]
#     new_positions[:,1] = (new_positions[:,1] - box_yrange[0]) % (box_yrange[1] - box_yrange[0]) + box_yrange[0]    
#     trajectory.append(new_positions)

# trajectory = np.array(trajectory)

In [42]:
# import ipywidgets as ipw

# fig = plt.figure(figsize=(7,6))
# @ipw.interact(
#     frame_idx=ipw.IntSlider(description='Frame', min=0, max=num_steps-1, value=0, continuous_update=False)
# )
# def plot(frame_idx):
#     frame_coords = trajectory[frame_idx]
#     plt.plot(frame_coords[:,0], frame_coords[:,1], '.')
    
#     std_radius = np.array(frame_coords).std(axis=0).mean()
#     circle = plt.Circle((0, 0), 2*std_radius, color='b', fill=False)
#     ax1.add_patch(circle)
    

#     ax1.set_aspect(1.)

#     x_coords = trajectory[frame_idx,:,0]
#     y_coords = trajectory[frame_idx,:,1]
#     H, x_edges, y_edges = np.histogram2d(x_coords, y_coords, bins=10, range= [[box_xrange[0], box_xrange[1]], [box_yrange[0], box_yrange[1]]], density=False) # compute histogram
#     H = H.T
#     ax2 = fig.add_subplot(1,3,2, sharex=ax1, sharey=ax1)
#     plt.imshow(H, origin='lower', interpolation='nearest', extent=[x_edges[0], x_edges[-1], y_edges[0], y_edges[-1]])
    
#     ax3 = fig.add_subplot(1,3,3, sharex=ax1, sharey=ax1, aspect='equal', xlim=x_edges[[0,-1]], ylim=y_edges[[0,-1]])
# #     im = mpl.image.NonUniformImage(ax, interpolation='bilinear')
# #     x_centers = (x_edges[:-1] + x_edges[1:]) / 2
# #     y_centers = (y_edges[:-1] + y_edges[1:]) / 2
# #     im.set_data(x_centers, y_centers, H)
#     plt.imshow(H, origin='lower', interpolation='bicubic', extent=[x_edges[0], x_edges[-1], y_edges[0], y_edges[-1]])
#     ax1.set_xlim(box_xrange)
#     ax1.set_ylim(box_yrange)
#     plt.show()
    
    
    

In [43]:
# define layout by gridspec
fig = plt.figure(constrained_layout=True, figsize=(7, 6))
gs = fig.add_gridspec(4,4)
ax1 = fig.add_subplot(gs[0:2,0:2])
ax2 = fig.add_subplot(gs[0:2,2:])
# # ax3 = fig.add_subplot(gs[-1,0:2])
ax3 = fig.add_subplot(gs[2:,2:])


trajectory =[] # trajectory of all dots

def plot_dots_circle(ax, frame_idx, show_traj):
    radius_factor = 2
    frame_coords = trajectory[frame_idx]
    ax.clear()
    ax.set_xlim(box_xrange)
    ax.set_ylim(box_yrange)
#     ax.set_aspect(1.)

    
    if show_traj:
        # show the path of the first particle up to current frame
        ax.plot(frame_coords[:,0], frame_coords[:,1], '.', alpha=0.5, zorder=1)
        ax.plot(trajectory[:frame_idx,0,0], trajectory[:frame_idx,0,1],linewidth=1, color='r', zorder=2, label='random path')
    else:
        ax.plot(frame_coords[:,0], frame_coords[:,1], '.',zorder=1)
    std_radius = np.array(frame_coords).std(axis=0).mean()
    circle = plt.Circle((0, 0), radius_factor*std_radius, color='k', fill=False, linestyle='dashed',zorder=3, label='mean radius, $r_{std}$')
    ax.add_patch(circle)
    ax.legend()

def plot_histogram(ax, frame_idx, is_continuous=False,):
    ax1.clear()
    x_coords = trajectory[frame_idx,:,0]
    y_coords = trajectory[frame_idx,:,1]
    H, x_edges, y_edges = np.histogram2d(x_coords, y_coords, bins=10, range= [[box_xrange[0], box_xrange[1]], [box_yrange[0], box_yrange[1]]], density=False) # compute histogram
    H = H.T
    if is_continuous:
        ax.imshow(H, origin='lower', interpolation='bicubic', extent=[x_edges[0], x_edges[-1], y_edges[0], y_edges[-1]])
    else:
        ax.imshow(H, origin='lower', interpolation='nearest', extent=[x_edges[0], x_edges[-1], y_edges[0], y_edges[-1]])
#         im = mpl.image.NonUniformImage(ax, interpolation='bilinear')
#         x_centers = (x_edges[:-1] + x_edges[1:]) / 2
#         y_centers = (y_edges[:-1] + y_edges[1:]) / 2
#         im.set_data(x_centers, y_centers, H)
    ax.set_xlim(x_edges[[0,-1]])
    ax.set_ylim(y_edges[[0,-1]])

    
def plot_distance(ax, frame_idx):
    ax.clear()
    ax.set_xlim(0,10)
    ax.set_ylim(0,100)
    ax.set_xlabel("distance from centre")
    ax.set_ylabel("frequency")
    
    frame_coords = trajectory[frame_idx]
    radii = np.sqrt((frame_coords**2).sum(axis=1))
    hist_data, hist_x = np.histogram(radii, bins=30)
    hist_coords = np.concatenate([np.array(list(zip(hist_x[:-1], hist_data))), np.array(list(zip(hist_x[1:], hist_data)))])
    hist_coords = hist_coords[hist_coords[:, 0].argsort(), :]
    ax.plot(hist_coords[:, 0], hist_coords[:, 1])

def run(change):
    '''run diffusion and store in trajectory'''
    global trajectory
    # initial coords
    random_step = max_slider.value # max stepsize
    coords = (np.random.random((10*N, 2)) - 0.5)*2 * random_step # Between -starting_radius and starting_radius
    coords = coords[(coords**2).sum(axis=1) < random_step**2][:N] # taking points in the circle 

    assert len(coords) == N # check if all points are in the circle ??why??

    # run simulation and store trajectory 
    trajectory = [coords]
    num_steps = steps_slider.value
    for i in range(num_steps):
        random_displacement = (np.random.random((N, 2)) - 0.5) * 2 * random_step
        new_positions = trajectory[-1] + random_displacement
        # Some points might have gone beyond the box.
        # I could either reflect them back as a hard wall, or just use PBC. For simplicity, I use PBC
        new_positions[:,0] = (new_positions[:,0] - box_xrange[0]) % (box_xrange[1] - box_xrange[0]) + box_xrange[0]
        new_positions[:,1] = (new_positions[:,1] - box_yrange[0]) % (box_yrange[1] - box_yrange[0]) + box_yrange[0]    
        trajectory.append(new_positions)
    trajectory = np.array(trajectory)
    # enable play and frame slider after the simulation run
    play.disabled = False
    frame_slider.disabled = False

def stop(change):
    ''' disable play widget and reset frame slider'''
    play.disabled = True
    frame_slider.value = 0
#     map_chkbox.disabled = traj_chkbox.value
#     traj_chkbox.disabled = map_chkbox.value
    

def plot_radii(ax, frame_idx):
    times = []
    std_radius = []
    ax.clear()
    # Do std-deviation for both x and y coords, at each timestep (axis=1 means on all points for that step)
    # then do mean on second axis, meaning (x_std + y_std)/2 at each timestep
    std_radius = np.array(trajectory).std(axis=1).mean(axis=1)

    ax.plot(std_radius**2)
    ax.plot(frame_idx, std_radius[frame_idx]**2, 'ro', label='current frame')
    ax.set_xlabel('time frame')
    ax.set_ylabel('$r_{std}^2$')
    ax.legend()

def plot_frame(change):
    ''' plot current frame for all axis'''
    frame_idx = frame_slider.value
    if map_chkbox.value:
        is_continuous = (func_drpdwn.value == 'Continuous')
        plot_histogram(ax1, frame_idx=frame_idx, is_continuous=is_continuous)
    else:
        plot_dots_circle(ax1, frame_idx=frame_idx, show_traj=traj_chkbox.value)
    
    plot_distance(ax2, frame_idx)
    plot_radii(ax3, frame_idx)
    
# link widgets
ipw.jslink((play, 'value'), (frame_slider, 'value'))
ipw.jslink((steps_slider, 'value'), (frame_slider,'max'))
frame_slider.observe(plot_frame, names='value', type='change')
# click run for simmulation and collect trajectory
run_btn.on_click(run)

# change simulation parameters will disable play and frame slider until finish run
n_slider.observe(stop, names='value', type='change')
max_slider.observe(stop, names='value', type='change')
steps_slider.observe(stop, names='value', type='change')
func_drpdwn.observe(stop, names='value', type='change')
traj_chkbox.observe(stop, names='value', type='change')
map_chkbox.observe(stop,names='vale', type='change')


### how to layout plot and widgets together?
display(ipw.VBox([func_drpdwn, n_slider, max_slider, steps_slider, traj_chkbox, map_chkbox]))
display(frame_slider)
display(ipw.HBox([run_btn, play]))


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

IntSlider(value=0, continuous_update=False, description='Frame index', disabled=True, max=10000, step=100)

In [44]:
# from matplotlib import animation

# fps = 25
# video_length_seconds = 10.
# num_video_frames = int(fps*video_length_seconds)

# fig_movie = pl.figure(figsize=(10,10))
# ax = pl.subplot(1,1,1)
# points_plot = pl.plot(trajectory[0][:,0], trajectory[0][:,1], '.')
# circle = pl.Circle((0, 0), 0., color='b', fill=False)
# ax.add_patch(circle)
# pl.xlim(box_xrange)
# pl.ylim(box_yrange)
# ax.set_aspect(1.)

# def update_frame(frame_idx):
#     global trajectory, points_plot, num_video_frames, num_steps, circle
    
#     trajectory_idx = frame_idx * (num_steps // num_video_frames)    
#     frame_coords = trajectory[trajectory_idx]
#     points_plot[0].set_data(frame_coords.T)

#     std_radius = np.array(frame_coords).std(axis=0).mean()
#     circle.set_radius(2*std_radius)
    
#     return points_plot[0], circle

# line_ani = animation.FuncAnimation(fig_movie, update_frame, frames=num_video_frames, blit=True)
# line_ani.save('diffusion.mp4', fps=fps)

In [45]:
# run(None)
# plt.figure()


# for frame_idx in [0, 1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 9999]:
#     frame_coords = trajectory[frame_idx]
#     radii = np.sqrt((frame_coords**2).sum(axis=1))
#     hist_data, hist_x = np.histogram(radii, bins=30)
#     hist_coords = np.concatenate([np.array(list(zip(hist_x[:-1], hist_data))), np.array(list(zip(hist_x[1:], hist_data)))])
#     hist_coords = hist_coords[hist_coords[:, 0].argsort(), :]
#     plt.plot(hist_coords[:, 0], hist_coords[:, 1])
# plt.xlim(0, 10)
# plt.xlabel("distance from centre")
# plt.ylabel("frequency")
# plt.show()

In [46]:
import pylab as pl
times = []
std_radius = []

pl.figure(figsize=(10,7))

# Do std-deviation for both x and y coords, at each timestep (axis=1 means on all points for that step)
# then do mean on second axis, meaning (x_std + y_std)/2 at each timestep
std_radius = np.array(trajectory).std(axis=1).mean(axis=1)
# TWICE the radius (to match the video)
pl.plot(2*std_radius)

## SLOWER VERSION
#for frame_idx, frame_coords in enumerate(tqdm.tqdm(trajectory)):
#    x_std = frame_coords[:, 0].std()
#    y_std = frame_coords[:, 1].std()
#    std = (x_std + y_std)/2.
#    times.append(frame_idx)
#    std_radius.append(std)
#pl.plot(times, std_radius)

pl.xlabel('time')
pl.ylabel('average radius')
pl.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

AxisError: axis 1 is out of bounds for array of dimension 1

In [ ]:
# test_play = ipw.Play(value=0, min=0, max=10, step=1, disabled=True)
# test_slider = ipw.IntSlider(value=0, min=0, max=10, step=1, disabled=True)
# ipw.jslink((test_play, 'value'), (test_slider, 'value'))
# test_chkbox = ipw.Checkbox(value=False, description='enable play')
# display(ipw.VBox([test_play, test_slider,test_chkbox]))
# def enable(change):
#     if test_chkbox.value:
#         test_play.disabled=False
#         test_slider.disabled=False
#     else:
#         test_play.disabled=True
#         test_slider.value=0
#         test_slider.disabled=True

# plot_f(None)
# test_chkbox.observe(enable, names='value', type='change')

In [ ]:
def plot_radii(ax, frame_idx):
    times = []
    std_radius = []
    ax.clear()
    # Do std-deviation for both x and y coords, at each timestep (axis=1 means on all points for that step)
    # then do mean on second axis, meaning (x_std + y_std)/2 at each timestep
    std_radius = np.array(trajectory).std(axis=1).mean(axis=1)

    ax.plot(std_radius)
    ax.plot(frame_idx, std_radius[frame_idx], 'ro', label='current frame')
    ax.set_xlabel('time frame')
    ax.set_ylabel('$r_{\text{std}}^2$')
    ax.legend()

fig, ax = plt.subplots()

run(None)
plot_radii(ax, 3)